In [2]:
import board
import time
import neopixel_spi as neopixel
from dataclasses import dataclass
import numpy as np
import webcolors
import threading
import copy
from colour import Color

NUM_PIXELS = 64

@dataclass
class Colors:
    # https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf
    GREEN = Color("green") # (0, 228, 0)
    YELLOW = Color("yellow") # different
    ORANGE = Color("orange")
    RED = Color("red")
    PURPLE = Color("purple")
    MAROON = Color("maroon")
    BLACK = Color()

all_colors = [Colors.GREEN, Colors.YELLOW, Colors.ORANGE, Colors.RED, Colors.PURPLE, Colors.MAROON]

# 24bits (rgb) * 1/640kHz = 30us = 0.030ms
# As per: https://learn.adafruit.com/adafruit-neopixel-uberguide/advanced-coding#faq-2894689
UPDATE_PER_PIXEL_MS = 0.0375


UPDATE_TIME = UPDATE_PER_PIXEL_MS * NUM_PIXELS + 0.050 # 50us of "latching time"
UPDATE_TIME

INTENSITY_MIN = 0
INTENSITY_MAX = 255

def rgb_to_hex(r=0, g=0, b=0):
    color_hex = (r << 16) + (g << 8) + b
    return color_hex

def test_rgb_to_hex():
    rgb_to_hex(g=228) == Color.GREEN


class LED(threading.Thread):

    def __init__(self):
        super().__init__()

        # Lock-protected color variable
        self._color = Colors.GREEN
        self._color_lock = threading.Lock()

        spi = board.SPI()
        self._pixels = neopixel.NeoPixel_SPI(spi, n=NUM_PIXELS, pixel_order=neopixel.GRB, auto_write=True, brightness=0.1)

    def run(self):
        while True:

            # Step 1: Get color value (lock-protected)
            with self._color_lock:
                color = copy.copy(self._color)

            # Breathe in:
            start = time.time()
            for i in range(255):
                color = Color(self._color, luminance=i/255)
                color_hex = int(color.hex_l[1:], 16)
                self._pixels.fill(color_hex) # 5.8ms per update
                time.sleep(4.2/1000)

            print(time.time() - start)

            # Breathe out:
            for i in reversed(range(255)):
                color = Color(self._color, luminance=i/255)
                color_hex = int(color.hex_l[1:], 16)
                print(color_hex)
                self._pixels.fill(color_hex) # 5.8ms per update
#                 time.sleep(4.2/1000)

            print(time.time() - start)

    def set_color(self, color: Color):
        with self._color_lock:
            self._color = color


In [12]:
c = Color('red')
c.set_luminance(0.5)

In [14]:
c.set_luminance?

In [5]:
led = LED()
led.set_color(Colors.MAROON)

In [6]:
led.start()

3.565229654312134
5.5001091957092285
3.975558280944824
5.858546733856201
3.783449172973633
5.678359508514404
3.801513671875
5.706780910491943
3.8217930793762207
5.7138261795043945
3.7782142162323
5.717108488082886
3.8962583541870117
5.804158449172974
3.8292102813720703
5.77311372756958
3.8805782794952393
5.7927350997924805
3.836637258529663
5.785445213317871
3.9165735244750977
5.870595455169678
3.8956260681152344
5.785776615142822
3.8249759674072266
5.749175548553467
3.8649332523345947
5.794469356536865
3.856262445449829
5.75568699836731
3.818563461303711
5.74039626121521
3.76332950592041
5.652517318725586
3.695970296859741
5.589428186416626
3.813014507293701
5.726164102554321
3.836772918701172
5.768089532852173
